## Watson discovery News - Updates based on today's Date - uncomment /comment relevant parts before running

#### This notebook is an example of how you can load Watson Discovery Search Results into db2.  It will normalise the results into several tables, making it ideal to analyse within many BI tools such as Cognos Analytics

### Install Watson Developer Cloud

In [ ]:
#!pip install pandas
#!pip install numpy
#!pip install --user pixiedust --upgrade
!pip install -I watson-developer-cloud==0.26.1
#print("Please restart the kernel and then run through the notebook again (skipping this cell).")

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
from datetime import datetime
import time
import requests

#### Below is the method to sign into the Watson discovery Service.  In this example we are using the Watson Discovery News Selection.  However, the same method can be applied to custom collections.  The max doc count is 50 documents.  If you need to retrieve more, you can append multiple subsets of data to one dataset - as below.  Here, i have used the combination of count and offset to select the subset of documents to retrieve

In [ ]:
import sys
import os
import json
from watson_developer_cloud import DiscoveryV1

CrawlDate = str(datetime.date(datetime.today()))
#CrawlDate = '2018-05-08'
NLQ = '{NATURAL LANGUAGE QUERY}'  #REPLACE {NATURAL LANGUAGE QUERY} with your natural language search i.e Transport in London
URLSearch = RBS.replace(" ", "%20")
discovery = DiscoveryV1(
  username="{USERNAME}", #REPLACE {USERNAME} with your username
  password="{PASSWORD}", #REPLACE {PASSWORD} with your password
  version="2017-11-07"
)

# USE THIS TO FILTER FOR EXAMPLE.....    'query': "division:'PBB' ,

#Below filter on news for today
qopts = {'query': "crawl_date:{},enriched_text.entities.text: {}".format(CrawlDate, NLQ),'count':50, 'offset':0}
qopts2 = {'query': "crawl_date:{},enriched_text.entities.text: {}".format(CrawlDate, NLQ),'count':50, 'offset':50}
qopts3 = {'query': "crawl_date:{},enriched_text.entities.text: {}".format(CrawlDate, NLQ),'count':50, 'offset':100}
qopts4 = {'query': "crawl_date:{},enriched_text.entities.text: {}".format(CrawlDate, NLQ),'count':50, 'offset':150}

#Below filter on all news in the last 2 months
#qopts = {'query': "enriched_text.entities.text: {}".format(NLQ),'count':50, 'offset':0}
#qopts2 = {'query': "enriched_text.entities.text: {}".format(NLQ),'count':50, 'offset':50}
#qopts3 = {'query': "enriched_text.entities.text: {}".format(NLQ),'count':50, 'offset':100}
#qopts4 = {'query': "enriched_text.entities.text: {}".format(NLQ),'count':50, 'offset':150}



#### Below is the section where we retrieve the data.  System is the Watson Discovery Collection and News Environment is the Watson Discovery News Enviornment.  For custom collections, please amend as appropitate. 

In [ ]:
newsz = discovery.query('system', 'news-en', qopts)
newsx = discovery.query('system', 'news-en', qopts2)
newsy = discovery.query('system', 'news-en', qopts3)
newsw = discovery.query('system', 'news-en', qopts4)


#### Below is the section where we package our data into data frames, then append all subsets of data into one dataframe.  We will finish with one dataframe which displays all the results 

In [ ]:
level1_df = pd.DataFrame()

newszdf = pd.io.json.json_normalize((newsz['results']),sep='_')
newsxdf = pd.io.json.json_normalize((newsx['results']),sep='_')
newsydf = pd.io.json.json_normalize((newsy['results']),sep='_')
newsvdf = pd.io.json.json_normalize((newsw['results']),sep='_')

level1_df = level1_df.append(newszdf)
level1_df = level1_df.append(newsxdf)
level1_df = level1_df.append(newsydf)
level1_df = level1_df.append(newsvdf)



#### Below is the section where we add additional columns to the dataframe - we reformat the publication date to a time based format as well.  I have then created a new dataframe which includes main index.  This is actually not required in this example, however, if you want to create multiple queries and use a 'FOR' statement to retrieve them all, you may want to have a new index but retain the original ones too - this is so you can join to a 'search masterfile' which would contain all searches

In [ ]:
#Load all search results data into a dataframe

level1 = pd.DataFrame()
Level1New = pd.DataFrame()
LevelNew2 = pd.DataFrame()
#ASSIGN LEVEL 1 HEADERS TO PLACES


level1_df ['PublicationDate'] = level1_df.publication_date#.apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))
level1_df ['CrawlDate'] = level1_df.crawl_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))
level1_df ['SearchQuery']= RBS    
level1_df['PrimaryKey'] = level1_df.index
    
level1 = level1.append(level1_df)

    
    
level1_df.reset_index(inplace=True)


Level1New = Level1New.append(level1)
LevelNew2 = Level1New.reset_index()
LevelNew2 ['PrimaryInd'] = LevelNew2.index



#### Here I am creating a new dataframe based on the existing one, then i am dropping columns that are not needed.  

In [ ]:
##refine Header Table


Level3New = LevelNew2
Level3New = LevelNew2
Level3New = Level3New.drop((['enriched_text_categories',
                             'enriched_text_concepts',
                             'enriched_text_entities',
                             'enriched_text_keywords',
                             'enriched_text_relations',
                             'enriched_text_semantic_roles',
                             'enriched_title_categories',
                             'enriched_title_entities',
                             'enriched_title_keywords',
                             'enriched_title_relations',
                             'enriched_title_concepts',
                             'crawl_date',
                             'extracted_metadata_filename',
                             'extracted_metadata_sha1',
                             'enriched_title_semantic_roles'
                            
                            
                            ]),1)
Level3New[['enriched_title_sentiment_document_score','enriched_text_sentiment_document_score']] = Level3New[['enriched_title_sentiment_document_score','enriched_text_sentiment_document_score']].apply(pd.to_numeric)
Level3New

#### Below, we are splitting the enriched columns into tables.  You will notice before that these columns had nested json.  This is how i deal with the results.

In [ ]:
#Detailed Analysis Tables

Level1New4 = pd.DataFrame(LevelNew2.index)
ConceptsHD = pd.DataFrame()
CategoriesHD = pd.DataFrame()
KeywordsHD = pd.DataFrame()
Level1Index = pd.DataFrame()
entitiesHD = pd.DataFrame()
SemanticHD = pd.DataFrame()

ConceptsHD = pd.DataFrame()
Level1Index = pd.DataFrame()
Level1Index ['ArrayIndex'] = Level1New4.index
CONCEPTS = Level1Index.ArrayIndex

#Concepts, categories, keywords, Entity and Relation Tables

for A in CONCEPTS:
    ROOTIND = A
    Concepts_Df = pd.io.json.json_normalize(LevelNew2.enriched_text_concepts[ROOTIND],sep='_') #the sep='_' is required because the default is a '.' seperator, which makes this difficult to query
    Concepts_Df['PrimaryKey'] = A
    Concepts_Df['ConceptKey']= Concepts_Df.index
    ConceptsHD = ConceptsHD.append(Concepts_Df)
  
    
    Categories_Df = pd.io.json.json_normalize(LevelNew2.enriched_text_categories[ROOTIND],sep='_')
    Categories_Df['PrimaryKey'] = A
    Categories_Df['CategoryKey']= Categories_Df.index
    CategoriesHD = CategoriesHD.append(Categories_Df)
 
    keywords_Df = pd.io.json.json_normalize(LevelNew2.enriched_text_keywords[ROOTIND],sep='_')
    keywords_Df['PrimaryKey'] = A
    keywords_Df['KeywordKey']= keywords_Df.index
    KeywordsHD = KeywordsHD.append(keywords_Df)
    
    entities_Df = pd.io.json.json_normalize(LevelNew2.enriched_text_entities[ROOTIND],sep='_')
    entities_Df['PrimaryKey'] = A
    entities_Df['EntityKey']= entities_Df.index
    entitiesHD = entitiesHD.append(entities_Df)
    
    Semantic_Df = pd.io.json.json_normalize(LevelNew2.enriched_text_semantic_roles[ROOTIND],sep='_')
    Semantic_Df['PrimaryKey'] = A
    Semantic_Df['SemanticKey']= Semantic_Df.index
    SemanticHD = SemanticHD.append(Semantic_Df)

Concepts_Df.reset_index(inplace=True)    
Categories_Df.reset_index(inplace=True)
keywords_Df.reset_index(inplace=True)
entities_Df.reset_index(inplace=True)
Semantic_Df.reset_index(inplace=True)

Categories_Table = CategoriesHD.reset_index()
Concepts_Table = ConceptsHD.reset_index()
Keywords_Table = KeywordsHD.reset_index()
Entities_Table = entitiesHD.reset_index()
Semantic_Table = SemanticHD.reset_index()

#### Here i am viewing the results of the sub tables

In [ ]:
Concepts_Table.head(10)

In [ ]:
Categories_Table.head(5)

In [ ]:
Keywords_Table.head(1)

In [ ]:
Entities_Table.head(3)

#### Here i am dropping unnessary Semantic Table columns, and creating new dataframes for the nested json within Semantic_Table.

In [ ]:
SemantecEntities = pd.DataFrame(Semantic_Table.object_entities).dropna (axis=0, how='all')
SemantecKeywords = pd.DataFrame(Semantic_Table.object_keywords).dropna (axis=0, how='all')
Semantic_Table2 = pd.DataFrame(Semantic_Table.drop((['object_entities','object_keywords','subject_keywords','subject_entities','subject_text']),1))
Semantic_Table2 ['PrimaryInd'] = Semantic_Table2.index                               
Semantic_Table2

#### This is how we populate new dataframes with the nested json held in the Semantic Table - Semantic Keywords and Semantic Entities

In [ ]:
#Detailed Semantic Tables

ObjectEntitiesHD = pd.DataFrame()
ObjectKeywordsHD = pd.DataFrame()
SemanticIndex = pd.DataFrame()
SemanticIndex ['ArrayIndex'] = SemantecEntities.index
SemanticIndex1 = pd.DataFrame()
SemanticIndex1 ['ArrayIndex'] = SemantecKeywords.index

SEMANTICDET = SemanticIndex.ArrayIndex

SEMANTICDET1 = SemanticIndex1.ArrayIndex

#ObjectEntities and Keywords

for A in SEMANTICDET:
    SEMIND = A
    ObjectEntities_Df = pd.io.json.json_normalize(Semantic_Table.object_entities[SEMIND],sep='_')
    ObjectEntities_Df['Semantic_Key'] = A
    #ObjectEntities_Df['ObjectEnt_Key']= ObjectEntities_Df.index
    ObjectEntitiesHD = ObjectEntitiesHD.append(ObjectEntities_Df)
    
                                                  
ObjectEntities_Df.reset_index(inplace=True)                                                  
SemanticsObjectEntitiesTable = ObjectEntitiesHD.reset_index()                                                  

for B in SEMANTICDET1:
    SEMIND1 = B
    ObjectKeywords_Df = pd.io.json.json_normalize(Semantic_Table.object_keywords[SEMIND1],sep='_')
    ObjectKeywords_Df['Semantic_Key'] = B
    ObjectKeywords_Df['Object_Key']= ObjectKeywords_Df.index
    ObjectKeywordsHD = ObjectKeywordsHD.append(ObjectKeywords_Df)
        
ObjectKeywords_Df.reset_index(inplace=True)
SemanticsKeywordsTable = ObjectKeywordsHD.reset_index()

In [ ]:
SemanticsObjectEntitiesTable = SemanticsObjectEntitiesTable.drop(('disambiguation_subtype'),1)



SemanticsObjectEntitiesTable.head(2)


In [ ]:
SemanticsKeywordsTable.head(2)

#### Here i have chosen to apply the same numeric number to every row in every table which is imported.  This is so i can later query on the batch that has been imported.  If you are updating tables with new data, you may want to delete the entire batch - this timestamp will allow you to do this in SQL

In [ ]:
#Add a timestamp to all tables
from datetime import datetime
import time
import requests
DateTime = datetime.today().strftime("%d%m%Y%H%M%S")
SemanticsKeywordsTable['TimeStamp'] = DateTime 

SemanticsObjectEntitiesTable['TimeStamp'] = DateTime 

Semantic_Table2['TimeStamp'] = DateTime
Entities_Table['TimeStamp'] = DateTime
Keywords_Table['TimeStamp'] = DateTime
Categories_Table['TimeStamp'] = DateTime
Concepts_Table['TimeStamp'] = DateTime
Level3New['TimeStamp'] = DateTime

Semantic_Table2.dtypes

#### Here i am adding a publication date and time based on the date and time field

In [ ]:
from datetime import datetime
import time

Level3New ['Publication_Date_and_Time'] = Level3New.publication_date.str.slice(0,19).apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'))
Level3New ['Publication_Date'] = Level3New.publication_date.str.slice(0, 10)
Level3New ['Publication_Time'] = Level3New.publication_date.str.slice(12, 19)

#Level3New['PrimaryKey'] = level3New.index

Level3New.head(20000)

#### This is where we reconstruct the header table - also to include additional columns such as YEAR, MONTH, DAY, MONTH NAME, WEEKDAY NUMBER AND WEEKDAY NAME.  Please note, you will see that the text is a substring (1st 200 characters) - the limit you can enter will depend on the column size of the field in db2.  if ibmdbpy automatically generates the tables, it will create tables based on information given from the first loaded record.  I would advise to load (autogenerate) initially to retrieve the tables, amend the suggested column sizes in db2 then delete all rows,  and then from that point onwards use the append section when you load actual data.

In [ ]:
import calendar
import datetime

HEADER = Level3New

HEADER.text = HEADER.text.str.slice(0, 600)

HEADER_2 = pd.DataFrame()
HEADER_2['PRIMARY_KEY']= HEADER.PrimaryKey
HEADER_2['DocID']= HEADER.id
HEADER_2['PUBLICATION_DATE']= HEADER.Publication_Date
HEADER_2['YEAR'] = HEADER.Publication_Date_and_Time.dt.year
HEADER_2['MONTH'] = HEADER.Publication_Date_and_Time.dt.month
HEADER_2['DAY'] = HEADER.Publication_Date_and_Time.dt.day
HEADER_2['MONTH_NAME'] =  HEADER_2['MONTH'].astype(str).str.zfill(2) + (' ') + HEADER_2['MONTH'].apply(lambda x: calendar.month_name[x])
HEADER_2['MONTH_NME'] =  HEADER_2['MONTH'].apply(lambda x: calendar.month_name[x])
HEADER_2['WEEKDAY_NUMBER'] = HEADER.Publication_Date_and_Time.dt.weekday
HEADER_2['WEEKDAY_NAME'] = HEADER_2['WEEKDAY_NUMBER'].apply(lambda x: calendar.day_name[x])
HEADER_2['Month_Number Str'] = HEADER_2['MONTH'].astype(str).str.zfill(2)
HEADER_2['PUBLICATION_DATE_AND_TIME']= HEADER.Publication_Date_and_Time
HEADER_2['author']= HEADER.author
HEADER_2['URL']= HEADER.url
HEADER_2['Title']= HEADER.title
HEADER_2['Forum Title']= HEADER.forum_title
HEADER_2['Source Typ']= HEADER.source_type
HEADER_2['Country']= HEADER.country
HEADER_2['text']= HEADER.text.str.slice(0, 200)
HEADER_2['Sentiment_Score']= HEADER.enriched_text_sentiment_document_score
HEADER_2['TimeStamp']= HEADER.TimeStamp
HEADER_2.head(10)#.text[0]

#### Here we are reconstructing the Entities table and slicing the text to add the first 200 characters - again, here you could do an initial load, alter the table properties for the text column and then reload using the update method 

In [ ]:
Entities_Table2 = pd.DataFrame()

Entities_Table2['index'] = Entities_Table.index
#Entities_Table2['Entity_Key'] = Entities_Table.EntityKey
Entities_Table2['Primary_Key'] = Entities_Table.PrimaryKey                           
#Entities_Table2['Count'] = Entities_Table.count                              
Entities_Table2['disambiguation_resource'] = Entities_Table.disambiguation_dbpedia_resource     
Entities_Table2['disambiguation_name'] = Entities_Table.disambiguation_name
Entities_Table2['disambiguation_subtype'] = Entities_Table.disambiguation_subtype.str[0]
Entities_Table2['relevance'] = Entities_Table.relevance                          
Entities_Table2['sentiment_label'] = Entities_Table.sentiment_label                     
Entities_Table2['sentiment_score'] = Entities_Table.sentiment_score                    
Entities_Table2['text'] = Entities_Table.text.str.slice(0,200)                                
Entities_Table2['type'] = Entities_Table.type                                
Entities_Table2['TimeStamp'] = Entities_Table.TimeStamp

Entities_Table2.head(5)

In [ ]:
Semantic_Table2.object_text = Semantic_Table2.object_text.str.slice(0, 200)
Semantic_Table2.sentence = Semantic_Table2.sentence.str.slice(0, 200)
#Semantic_Table2.subject_text = Semantic_Table2.subject_text.str.slice(0, 200)

Semantic_Table2

#### To ensure the order of columns is the same as what is in the database, i have set the order into new dataframes (with the exeption of entities and Header which have already been reconstructed)

In [ ]:
Categories_TableDB2 = pd.DataFrame()
Semantic_TableDB2 = pd.DataFrame()
Concepts_TableDB2 = pd.DataFrame()
Keywords_TableDB2 = pd.DataFrame()

#Reorder Categories Table

Categories_TableDB2['index'] = Categories_Table.index
Categories_TableDB2['label'] = Categories_Table.label
Categories_TableDB2['score'] = Categories_Table.score
Categories_TableDB2['PrimaryKey'] = Categories_Table.PrimaryKey
Categories_TableDB2['CategoryKey'] = Categories_Table.CategoryKey
Categories_TableDB2['TimeStamp'] = Categories_Table.TimeStamp


#Reorder Concepts Table

Concepts_TableDB2['index'] = Concepts_Table.index
Concepts_TableDB2['dbpedia_resource'] = Concepts_Table.dbpedia_resource
Concepts_TableDB2['relevance'] = Concepts_Table.relevance
Concepts_TableDB2['text'] = Concepts_Table.text
Concepts_TableDB2['PrimaryKey'] = Concepts_Table.PrimaryKey
Concepts_TableDB2['ConceptKey'] = Concepts_Table.ConceptKey
Concepts_TableDB2['TimeStamp'] = Concepts_Table.TimeStamp

#Reorder Semantic Table

Semantic_TableDB2['index'] = Semantic_Table2.index
Semantic_TableDB2['PrimaryKey'] = Semantic_Table2.PrimaryKey
Semantic_TableDB2['SemanticKey'] = Semantic_Table2.SemanticKey
Semantic_TableDB2['action_normalized'] = Semantic_Table2.action_normalized
Semantic_TableDB2['action_text'] = Semantic_Table2.action_text
Semantic_TableDB2['action_verb_negated'] = Semantic_Table2.action_verb_negated
Semantic_TableDB2['action_verb_tense'] = Semantic_Table2.action_verb_tense
Semantic_TableDB2['action_verb_text'] = Semantic_Table2.action_verb_text
Semantic_TableDB2['object_text'] = Semantic_Table2.object_text
Semantic_TableDB2['sentence'] = Semantic_Table2.sentence
Semantic_TableDB2['PrimaryInd'] = Semantic_Table2.PrimaryInd
Semantic_TableDB2['TimeStamp'] = Semantic_Table2.TimeStamp

#Reorder Keywords Table

Keywords_TableDB2['index']= Keywords_Table.index
Keywords_TableDB2['relevance']= Keywords_Table.relevance
Keywords_TableDB2['sentiment_label']= Keywords_Table.sentiment_label
Keywords_TableDB2['sentiment_score']= Keywords_Table.sentiment_score
Keywords_TableDB2['text']= Keywords_Table.text
Keywords_TableDB2['PrimaryKey']= Keywords_Table.PrimaryKey
Keywords_TableDB2['KeywordKey']= Keywords_Table.KeywordKey
Keywords_TableDB2['TimeStamp']= Keywords_Table.TimeStamp


### LOAD DATA INTO DB2, the first section updates existing tables, and the 2nd section Drops and Recreates tables based on the dataframe properties  Use second section for inital load and first section for updates (which is the one you would use for ongoing document retrieval

In [ ]:
import ibmdbpy
from ibmdbpy import IdaDataBase

#REPLACE HOSTNAME, USERNAME AND PASSWORD WITH your db2 credentials

idadb = IdaDataBase(dsn='DASHDB;Database=BLUDB;Hostname={HOSTNAME};Port=50000;PROTOCOL=TCPIP;UID={USERNAME};PWD={PASSWORD}',autocommit=True, verbose=False)

#  1ST SECTION USE THIS TO UPDATE TABLES (un comment when you use this)


#NEWS_KEYWORDS_TABLE = IdaDataFrame(idadb, 'BLUADMIN.NEWS_KEYWORDS_TABLE')
#NEWS_CATEGORIES_TABLE = IdaDataFrame(idadb, 'BLUADMIN.NEWS_CATEGORIES_TABLE')
#NEWS_SEMANTIC_TABLE = IdaDataFrame(idadb, 'BLUADMIN.NEWS_SEMANTIC_TABLE')
#NEWS_ENTITIES_TABLE = IdaDataFrame(idadb, 'BLUADMIN.NEWS_ENTITIES_TABLE')
#NEWS_CONCEPTS_TABLE = IdaDataFrame(idadb, 'BLUADMIN.NEWS_CONCEPTS_TABLE')
#NEWS_HEADER_TABLE = IdaDataFrame(idadb, 'BLUADMIN.NEWS_HEADER_TABLE')

#idadb.append(NEWS_KEYWORDS_TABLE, Keywords_TableDB2, maxnrow=None)
#idadb.append(NEWS_CATEGORIES_TABLE, Categories_TableDB2, maxnrow=None)
#idadb.append(NEWS_SEMANTIC_TABLE, Semantic_TableDB2, maxnrow=None)
#idadb.append(NEWS_ENTITIES_TABLE, Entities_Table2, maxnrow=None)
#idadb.append(NEWS_CONCEPTS_TABLE, Concepts_TableDB2, maxnrow=None)
#idadb.append(NEWS_HEADER_TABLE, HEADER_2, maxnrow=None)


########################################################################################

# 2ND SECTION USE THIS TO DROP AND REBUILD TABLES - NOTE ORIGINAL TABLES WILL BE DELETED (uncomment when you use this)

#idadf1 = idadb.as_idadataframe(Keywords_Table, "NEWS_KEYWORDS_TABLE", clear_existing=True)
#idadf2 = idadb.as_idadataframe(Categories_Table, "NEWS_CATEGORIES_TABLE", clear_existing=True)
#idadf8 = idadb.as_idadataframe(Semantic_Table2, "NEWS_SEMANTIC_TABLE", clear_existing=True)
#idadf9 = idadb.as_idadataframe(Entities_Table2, "NEWS_ENTITIES_TABLE", clear_existing=True)
#idadf4 = idadb.as_idadataframe(Concepts_Table, "NEWS_CONCEPTS_TABLE", clear_existing=True)
#idadf7 = idadb.as_idadataframe(HEADER_2, "NEWS_HEADER_TABLE", clear_existing=True)

########################################################################################


#Current WIP - This section still needs refinement
#idadf3 = idadb.as_idadataframe(SemanticsObjectEntitiesTable, "NEWS_OBJ_ENTITIES_TABLE", clear_existing=True)
#idadf5 = idadb.as_idadataframe(SemanticsKeywordsTable, "NEWS_OBJ_KEYWORDS_TABLE", clear_existing=True)

idadb.close()

# Tables in db2

In [ ]:
IdaDataFrame(idadb, 'BLUADMIN.NEWS_KEYWORDS_TABLE').dtypes#head()


In [ ]:
IdaDataFrame(idadb, 'BLUADMIN.NEWS_CATEGORIES_TABLE').dtypes#head()


In [ ]:
Categories_TableDB2#head()

In [ ]:
IdaDataFrame(idadb, 'BLUADMIN.NEWS_SEMANTIC_TABLE').dtypes#head()


In [ ]:
IdaDataFrame(idadb, 'BLUADMIN.NEWS_ENTITIES_TABLE').dtypes#head()


In [ ]:
IdaDataFrame(idadb, 'BLUADMIN.NEWS_CONCEPTS_TABLE').dtypes#head()


In [ ]:
IdaDataFrame(idadb, 'BLUADMIN.NEWS_HEADER_TABLE').dtypes#head()

#### please contact becky.oconnor@uk.ibm.com if you have any questions on the above.